## Import libraries

In [1]:
from google.colab import drive
from pathlib import Path
from matplotlib import pyplot as plt 
import pandas as pd
import numpy as np
import time
import os
import csv
import concurrent.futures

## Utility functions

### Create annot and load descriptors

In [2]:
def create_annot(path):
  image_list = list(Path(path).glob('*/*.jpg'))
  # the identity name is in the path (the name of the parent directory)
  names_list = [i.parent.name for i in image_list]  # get the identity of each image
  # keep info in a pandas DataFrame
  annot = pd.DataFrame({'identity': names_list, 'image_path': image_list})
  return annot

def concatenate_annots(list_of_paths):
  concat_annot = pd.DataFrame()
  with concurrent.futures.ThreadPoolExecutor() as executor:
    annots = [executor.submit(create_annot, path) for path in list_of_paths]
    for annot in annots:
      new_annot = annot.result()
      concat_annot = concat_annot.append(new_annot, ignore_index = True)
    return concat_annot

In [3]:
def load_descriptors(path):
  with open(path, 'rb') as file:
    return np.load(file)

def concatenate_descriptors(list_of_paths):
  concat_descriptors = None
  with concurrent.futures.ThreadPoolExecutor() as executor:
    descriptors = [executor.submit(load_descriptors, path) for path in list_of_paths]
    for descriptor in descriptors:
      new_descriptor = descriptor.result()
      if concat_descriptors is None:
        concat_descriptors = new_descriptor
      else:
        concat_descriptors = np.concatenate([concat_descriptors, new_descriptor])
    return concat_descriptors

### Create pivots

In [4]:
def generate_pivots(descriptors, n, strategy="rnd"):
  if strategy == "kMED":
    kmedoids = sklearn_extra.cluster.KMedoids(n_clusters=n).fit(descriptors)
    return kmedoids.cluster_centers_
  if strategy != "rnd":
    print(strategy, "was not implemented. Random pivots were returned")
  pivots_id = np.random.choice(np.arange(len(descriptors)), size=n)
  return descriptors[pivots_id]

def generate_list_of_pivots(descriptors, t, n, strategy="rnd"):
  list_of_pivots = []
  with concurrent.futures.ThreadPoolExecutor() as executor:
    pivots = [executor.submit(generate_pivots, descriptors, n, strategy) for i in range(t)]
    for pivot in concurrent.futures.as_completed(pivots):
      new_pivot = pivot.result()
      list_of_pivots.append(new_pivot)
    return list_of_pivots

### Save test results

In [5]:
def save_results(dir, file_name, results):
  with open(os.path.join(dir, file_name +".csv"), 'w') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(["CLASS", "AP", "QUERY TIME"])
    # write the data
    for r in results:
      writer.writerow(r) 

## Test Performance

In [6]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Create annot and load descriptors for the database

In [8]:
db_annot = concatenate_annots(['/content/drive/MyDrive/CV_Birds/train', '/content/drive/MyDrive/CV_Birds/mirflickr25k'])
db_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/train/AFRICAN ...
...,...,...
72327,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72328,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72329,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...
72330,mirflickr,/content/drive/MyDrive/CV_Birds/mirflickr25k/m...


In [9]:
db_descriptors = concatenate_descriptors(['/content/drive/MyDrive/CV_Birds/features/training/AutoEncoder/512to128withPace64_feature_extraction.npy','/content/drive/MyDrive/CV_Birds/features/distractor/AutoEncoder/512to128withPace64_feature_extraction.npy'])
db_descriptors.shape

(72332, 128)

### Create annot and load descriptors for the test set

In [10]:
query_annot = create_annot('/content/drive/MyDrive/CV_Birds/test')
query_annot

,identity,image_path
0,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
1,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
2,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
3,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
4,AFRICAN CROWNED CRANE,/content/drive/MyDrive/CV_Birds/test/AFRICAN C...
...,...,...
1620,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1621,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1622,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...
1623,YELLOW HEADED BLACKBIRD,/content/drive/MyDrive/CV_Birds/test/YELLOW HE...


In [11]:
query_descriptors = load_descriptors('/content/drive/MyDrive/CV_Birds/features/test/AutoEncoder/512to128withPace64_feature_extraction.npy')
query_descriptors.shape

(1625, 128)

To run our tests we select only the first image of each species within the test set. Please note that within the test set we have 5 images per species.

In [12]:
queries_indexes = [x for x in range(325*5) if x%5 == 0]

### Create PP-Index

In [13]:
def get_descriptor_from_id(id_object):
  return db_descriptors[id_object]

In [14]:
%cd "/content/drive/MyDrive/CV_Birds/Notebooks/PP-Index"
%run PPIndex.ipynb

/content/drive/.shortcut-targets-by-id/1rI5YNBuaSlCB__w522WEkHjw-nFuvIo0/CV_Birds/Notebooks/PP-Index


In [15]:
# generate pivots
pivots = generate_pivots(db_descriptors, 40, "rnd")

In [16]:
# cosine tree
cosine_tree = PrefixTree(pivots, length=3, distance_metric='cosine', base_directory="/content/cosine", tree_file='tree_structure')
if cosine_tree.is_empty():
  cosine_tree.insert_objects_into_tree(range(len(db_descriptors)))
  cosine_tree.save()

[WARNING]: Tree is still empty
Inserting objects into tree_structure ...
--- Insertion into tree tree_structure completed: 79.598 seconds ---


In [46]:
!cp /content/cosine/tree* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/tree/cosine/

In [17]:
# euclidean tree
euclidean_tree = PrefixTree(pivots, length=3, distance_metric='euclidean', base_directory="/content/euclidean", tree_file='tree_structure')
if euclidean_tree.is_empty():
  euclidean_tree.insert_objects_into_tree(range(len(db_descriptors)))
  euclidean_tree.save()

[WARNING]: Tree is still empty
Inserting objects into tree_structure ...
--- Insertion into tree tree_structure completed: 57.391 seconds ---


In [48]:
!cp /content/euclidean/tree* /content/drive/MyDrive/CV_Birds/indexes/feature_extraction/tree/euclidean/

### Compute mAP

In [ ]:
birds_db = db_annot.loc[db_annot['identity'] != 'mirflickr']
counts = birds_db.groupby('identity').count()
print("Minimum number of images per species:", int(counts.min()))
print("Maximum number of images per species:", int(counts.max()))
print("Average number of images:", float(counts.sum()/325))

Minimum number of images per species: 116
Maximum number of images per species: 249
Average number of images: 145.63692307692307


Since at most we have 249 images per species, we use $n=250$.

In [18]:
n = 250

The formula for Average Precision is the following:

> $AP@n=\frac{1}{GTP}\sum_{k=1}^{n}P@k×rel@k$

where $GTP$ refers to the total number of ground truth positives, $n$ refers to the total number of images we are interested in, $P@k$ refers to the precision@k and $rel@k$ is a relevance function. 

The relevance function is an indicator function which equals 1 if the document at rank $k$ is relevant and equals to 0 otherwise.

In [19]:
def compute_ap(query_index, retrieved_ids):
  query_identity = query_annot['identity'][query_index]
  print(query_index//5, query_identity)
  GTP = len(db_annot.loc[db_annot['identity'] == query_identity])
  relevant = 0
  precision_summation = 0
  for k, id in enumerate(retrieved_ids):
    if db_annot['identity'][id] == query_identity: # relevant result
      relevant = relevant + 1
      precision_at_k = relevant/(k+1)
      precision_summation = precision_summation + precision_at_k
  return (query_identity, precision_summation/GTP)

For each query, $Q$, we can calculate a corresponding $AP$. Then, the $mAP$ is simply the mean of all the queries that were made.
> $mAP = \frac{1}{N}\sum_{i=1}^{N}AP_i$

In our case, $N=325$ (one query per species)

#### Simple tree

##### Cosine

In [20]:
def cosine_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_tree.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [21]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [22]:
aps

[('AFRICAN CROWNED CRANE', 0.8983766080919565, 0.4548361301422119),
 ('AFRICAN FIREFINCH', 0.15527042545848052, 0.4956996440887451),
 ('ALBATROSS', 0.20284442331134417, 0.5065391063690186),
 ('ALEXANDRINE PARAKEET', 0.43248825670048013, 0.5434727668762207),
 ('AMERICAN AVOCET', 0.39106145251396646, 0.9156432151794434),
 ('AMERICAN BITTERN', 0.11764705882352941, 0.8979368209838867),
 ('AMERICAN COOT', 0.1518987341772152, 1.136918067932129),
 ('AMERICAN GOLDFINCH', 0.6699140647248163, 1.2855987548828125),
 ('AMERICAN KESTREL', 0.05213675213675214, 0.841376543045044),
 ('AMERICAN PIPIT', 0.21179580416827434, 0.5806264877319336),
 ('AMERICAN REDSTART', 0.04794422994536199, 0.5580406188964844),
 ('ANHINGA', 0.12342195263755495, 0.48948001861572266),
 ('ANNAS HUMMINGBIRD', 0.21228276115968173, 0.49167394638061523),
 ('ANTBIRD', 0.006666666666666667, 1.0346896648406982),
 ('ARARIPE MANAKIN', 0.5861593685673389, 0.6903254985809326),
 ('ASIAN CRESTED IBIS', 0.7313917483480811, 0.653610467910766

In [23]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [24]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.25970062955658474
avg. query time:  0.5670585705683782


In [25]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_tree_cosine_results', aps)

##### Euclidean

In [26]:
def euclidean_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_tree.find_nearest_neighbors(query_descriptors[query_index], n)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [27]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [28]:
aps

[('AFRICAN CROWNED CRANE', 0.372961646429798, 0.5217764377593994),
 ('AFRICAN FIREFINCH', 0.030114484818317574, 0.6124753952026367),
 ('ALBATROSS', 0.34330931450527524, 0.526547908782959),
 ('ALEXANDRINE PARAKEET', 0.22282236957495705, 0.5077245235443115),
 ('AMERICAN AVOCET', 0.3963500931098697, 0.48508763313293457),
 ('AMERICAN BITTERN', 0.39281316170014985, 0.5015208721160889),
 ('AMERICAN COOT', 0.23019691325056788, 0.5133183002471924),
 ('AMERICAN GOLDFINCH', 0.1397839585661847, 0.48957324028015137),
 ('AMERICAN KESTREL', 0.08589620952153969, 0.5074584484100342),
 ('AMERICAN PIPIT', 0.13521425749759228, 0.5262806415557861),
 ('AMERICAN REDSTART', 0.05040258832183022, 0.46797704696655273),
 ('ANHINGA', 0.1257495201696515, 0.48909902572631836),
 ('ANNAS HUMMINGBIRD', 0.1434214703566622, 0.45331263542175293),
 ('ANTBIRD', 0.006666666666666667, 0.5034844875335693),
 ('ARARIPE MANAKIN', 0.4944729711154132, 0.5310707092285156),
 ('ASIAN CRESTED IBIS', 0.301867468720908, 0.62929916381835

In [29]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [30]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.18628864249955177
avg. query time:  0.5239899312532865


In [31]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_tree_euclidean_results', aps)

#### Tree with query perturbation

##### Cosine

In [34]:
def cosine_pert_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = cosine_tree.find_nearest_neighbors_with_query_perturbation(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [35]:
aps = []
for query_index in queries_indexes:
  aps.append(cosine_pert_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [36]:
aps

[('AFRICAN CROWNED CRANE', 0.8753959658357497, 1.182852029800415),
 ('AFRICAN FIREFINCH', 0.23715278687471106, 1.4481940269470215),
 ('ALBATROSS', 0.5551205947350226, 1.2115638256072998),
 ('ALEXANDRINE PARAKEET', 0.596440951274926, 1.2095839977264404),
 ('AMERICAN AVOCET', 0.912264553363132, 1.3309106826782227),
 ('AMERICAN BITTERN', 0.6167183063635712, 1.1174066066741943),
 ('AMERICAN COOT', 0.6707091615140337, 1.2282366752624512),
 ('AMERICAN GOLDFINCH', 0.6739391852088064, 1.3402578830718994),
 ('AMERICAN KESTREL', 0.11144776069950248, 1.177347183227539),
 ('AMERICAN PIPIT', 0.6598746243889909, 1.0783860683441162),
 ('AMERICAN REDSTART', 0.063405250864163, 1.2559585571289062),
 ('ANHINGA', 0.13021126111661308, 1.1377477645874023),
 ('ANNAS HUMMINGBIRD', 0.46512595219753017, 1.0587990283966064),
 ('ANTBIRD', 0.01873921746168937, 1.2111926078796387),
 ('ARARIPE MANAKIN', 0.623808471345393, 1.29520845413208),
 ('ASIAN CRESTED IBIS', 0.7045838449037216, 1.1581757068634033),
 ('BALD EAG

In [37]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [38]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.35689121111371486
avg. query time:  1.2134827239696797


In [39]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_pert_tree_cosine_results', aps)

##### Euclidean

In [40]:
def euclidean_pert_tree_queries(query_index, n):
  start_time = time.time()
  ids, distances = euclidean_tree.find_nearest_neighbors_with_query_perturbation(query_descriptors[query_index], n, perturbations=3)
  end_time = time.time()
  ids = ids.tolist()
  return compute_ap(query_index, ids) + (end_time - start_time,)

In [41]:
aps = []
for query_index in queries_indexes:
  aps.append(euclidean_pert_tree_queries(query_index, n))

0 AFRICAN CROWNED CRANE
1 AFRICAN FIREFINCH
2 ALBATROSS
3 ALEXANDRINE PARAKEET
4 AMERICAN AVOCET
5 AMERICAN BITTERN
6 AMERICAN COOT
7 AMERICAN GOLDFINCH
8 AMERICAN KESTREL
9 AMERICAN PIPIT
10 AMERICAN REDSTART
11 ANHINGA
12 ANNAS HUMMINGBIRD
13 ANTBIRD
14 ARARIPE MANAKIN
15 ASIAN CRESTED IBIS
16 BALD EAGLE
17 BALD IBIS
18 BALI STARLING
19 BALTIMORE ORIOLE
20 BANANAQUIT
21 BANDED BROADBILL
22 BANDED PITA
23 BAR-TAILED GODWIT
24 BARN OWL
25 BARN SWALLOW
26 BARRED PUFFBIRD
27 BAY-BREASTED WARBLER
28 BEARDED BARBET
29 BEARDED BELLBIRD
30 BEARDED REEDLING
31 BELTED KINGFISHER
32 BIRD OF PARADISE
33 BLACK & YELLOW bROADBILL
34 BLACK BAZA
35 BLACK FRANCOLIN
36 BLACK SKIMMER
37 BLACK SWAN
38 BLACK TAIL CRAKE
39 BLACK THROATED BUSHTIT
40 BLACK THROATED WARBLER
41 BLACK VULTURE
42 BLACK-CAPPED CHICKADEE
43 BLACK-NECKED GREBE
44 BLACK-THROATED SPARROW
45 BLACKBURNIAM WARBLER
46 BLONDE CRESTED WOODPECKER
47 BLUE COAU
48 BLUE GROUSE
49 BLUE HERON
50 BLUE THROATED TOUCANET
51 BOBOLINK
52 BORNEAN BRI

In [42]:
aps

[('AFRICAN CROWNED CRANE', 0.264790175123263, 1.2415854930877686),
 ('AFRICAN FIREFINCH', 0.117543721270072, 1.2622559070587158),
 ('ALBATROSS', 0.37424162346934203, 1.1324982643127441),
 ('ALEXANDRINE PARAKEET', 0.22855610791069775, 1.160752296447754),
 ('AMERICAN AVOCET', 0.6883727673566147, 1.1206820011138916),
 ('AMERICAN BITTERN', 0.5389891537425205, 1.4363987445831299),
 ('AMERICAN COOT', 0.31687036884821695, 1.2040462493896484),
 ('AMERICAN GOLDFINCH', 0.1761356579268098, 1.4323358535766602),
 ('AMERICAN KESTREL', 0.07256684972404036, 1.1680097579956055),
 ('AMERICAN PIPIT', 0.49135825672672767, 1.127394437789917),
 ('AMERICAN REDSTART', 0.04446679795389465, 1.0716938972473145),
 ('ANHINGA', 0.1386312554040016, 1.2095062732696533),
 ('ANNAS HUMMINGBIRD', 0.15056371612584663, 1.0639770030975342),
 ('ANTBIRD', 0.02269486699880528, 1.1607472896575928),
 ('ARARIPE MANAKIN', 0.4691383894548818, 1.251873254776001),
 ('ASIAN CRESTED IBIS', 0.42284935697577986, 1.1974883079528809),
 ('B

In [43]:
ap_at_n = np.array([ap[1] for ap in aps])
query_time = np.array(([ap[2] for ap in aps]))

In [44]:
mAP_at_n = np.mean(ap_at_n, axis=0)
avg_query_time = np.mean(query_time, axis=0)
print("mAP:", mAP_at_n)
print("avg. query time: ", avg_query_time)

mAP: 0.24600120549429103
avg. query time:  1.1898033450200007


In [45]:
save_results('/content/drive/MyDrive/CV_Birds/performance/fine_tuning/index/AutoEncoder', 'AE_FE_pert_tree_euclidean_results', aps)